In [61]:
using Pkg

In [2]:
using LinearAlgebra

In [3]:
using DuckDB

In [4]:
using DataFrames

In [16]:
using BenchmarkTools

In [4]:
con = DBInterface.connect(DuckDB.DB, "dbfile1")

DuckDB.DB("dbfile1")

In [41]:
n=1000000

1000000

In [42]:
df = DataFrame(a = [Int32(x) for x in 1:n], b = [rand() for x in 1:n]);

In [43]:
df[!,"c"] .= convert.(Int32, 100 .* floor.(df[!,"a"] .// 100))

1000000-element Vector{Int32}:
       0
       0
       0
       0
       0
       0
       0
       0
       0
       0
       ⋮
  999900
  999900
  999900
  999900
  999900
  999900
  999900
  999900
 1000000

In [44]:
df

Row,a,b,c
,Int32,Float64,Int32
1,1,0.953131,0
2,2,0.296321,0
3,3,0.897179,0
4,4,0.144067,0
5,5,0.784014,0
6,6,0.346108,0
7,7,0.205078,0
8,8,0.60435,0
9,9,0.77532,0


In [45]:
DuckDB.register_data_frame(con, df, "my_df")

In [49]:
@btime results = DuckDB.execute(con, "CREATE OR REPLACE TABLE DATA1 AS SELECT * FROM my_df")

  61.098 ms (6921 allocations: 209.05 KiB)


1×1 DataFrame
 Row │ Count   
     │ Int64?  
─────┼─────────
   1 │ 1000000

In [66]:
DuckDB.execute(con, "SELECT * FROM  DATA1 LIMIT 3")

3×3 DataFrame
 Row │ a       b         c      
     │ Int32?  Float64?  Int32? 
─────┼──────────────────────────
   1 │      1  0.953131       0
   2 │      2  0.296321       0
   3 │      3  0.897179       0

In [72]:
@time DuckDB.execute(con, "COPY (SELECT * FROM DATA1) TO 'data1.parquet' (FORMAT PARQUET);")

  0.064812 seconds (16 allocations: 656 bytes)


1×1 DataFrame
 Row │ Count   
     │ Int64?  
─────┼─────────
   1 │ 1000000

In [77]:
@time DuckDB.execute(con, "SELECT * FROM 'data1.parquet' ;")

  0.014898 seconds (24 allocations: 816 bytes)


1000000×3 DataFrame
     Row │ a        b         c       
         │ Int32?   Float64?  Int32?  
─────────┼────────────────────────────
       1 │       1  0.953131        0
       2 │       2  0.296321        0
       3 │       3  0.897179        0
       4 │       4  0.144067        0
       5 │       5  0.784014        0
       6 │       6  0.346108        0
       7 │       7  0.205078        0
       8 │       8  0.60435         0
    ⋮    │    ⋮        ⋮         ⋮
  999994 │  999994  0.598799   999900
  999995 │  999995  0.666      999900
  999996 │  999996  0.951518   999900
  999997 │  999997  0.415549   999900
  999998 │  999998  0.588543   999900
  999999 │  999999  0.45919    999900
 1000000 │ 1000000  0.135341  1000000
                   999985 rows omitted

In [81]:
@time DuckDB.execute(con, "SELECT * FROM DATA1 where c = 999900;")

  0.000328 seconds (24 allocations: 816 bytes)


100×3 DataFrame
 Row │ a       b          c      
     │ Int32?  Float64?   Int32? 
─────┼───────────────────────────
   1 │ 999900  0.426579   999900
   2 │ 999901  0.710358   999900
   3 │ 999902  0.17435    999900
   4 │ 999903  0.98364    999900
   5 │ 999904  0.292794   999900
   6 │ 999905  0.320691   999900
   7 │ 999906  0.117782   999900
   8 │ 999907  0.0505681  999900
  ⋮  │   ⋮         ⋮        ⋮
  94 │ 999993  0.303132   999900
  95 │ 999994  0.598799   999900
  96 │ 999995  0.666      999900
  97 │ 999996  0.951518   999900
  98 │ 999997  0.415549   999900
  99 │ 999998  0.588543   999900
 100 │ 999999  0.45919    999900
                  85 rows omitted

In [75]:
@time DuckDB.execute(con, "COPY (SELECT * FROM DATA1) TO 'data1.csv' (FORMAT csv);")

  0.169202 seconds (16 allocations: 656 bytes)


1×1 DataFrame
 Row │ Count   
     │ Int64?  
─────┼─────────
   1 │ 1000000

In [79]:
@time DuckDB.execute(con, "SELECT * FROM 'data1.csv';")

  0.113505 seconds (24 allocations: 816 bytes)


1000000×3 DataFrame
     Row │ column0  column1   column2 
         │ Int64?   Float64?  Int64?  
─────────┼────────────────────────────
       1 │       1  0.953131        0
       2 │       2  0.296321        0
       3 │       3  0.897179        0
       4 │       4  0.144067        0
       5 │       5  0.784014        0
       6 │       6  0.346108        0
       7 │       7  0.205078        0
       8 │       8  0.60435         0
    ⋮    │    ⋮        ⋮         ⋮
  999994 │  999994  0.598799   999900
  999995 │  999995  0.666      999900
  999996 │  999996  0.951518   999900
  999997 │  999997  0.415549   999900
  999998 │  999998  0.588543   999900
  999999 │  999999  0.45919    999900
 1000000 │ 1000000  0.135341  1000000
                   999985 rows omitted

In [74]:
@time DuckDB.execute(con, "COPY (SELECT * FROM 'california_housing_test.csv') TO 'calhousetest.parquet' (FORMAT PARQUET);")

  0.044730 seconds (16 allocations: 656 bytes)


1×1 DataFrame
 Row │ Count  
     │ Int64? 
─────┼────────
   1 │   3000

In [11]:
results = DuckDB.execute(con, "CREATE OR REPLACE TABLE CALHOUSETST AS SELECT * FROM 'california_housing_test.csv' ")

1×1 DataFrame
 Row │ Count  
     │ Int64? 
─────┼────────
   1 │   3000

In [12]:
DuckDB.execute(con, "SELECT * FROM  CALHOUSETST")

3000×9 DataFrame
  Row │ longitude  latitude  housing_median_age  total_rooms  total_bedrooms   ⋯
      │ Float64?   Float64?  Float64?            Float64?     Float64?         ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │   -122.05     37.37                27.0       3885.0           661.0   ⋯
    2 │   -118.3      34.26                43.0       1510.0           310.0
    3 │   -117.81     33.78                27.0       3589.0           507.0
    4 │   -118.36     33.82                28.0         67.0            15.0
    5 │   -119.67     36.33                19.0       1241.0           244.0   ⋯
    6 │   -119.56     36.51                37.0       1018.0           213.0
    7 │   -121.43     38.63                43.0       1009.0           225.0
    8 │   -120.65     35.48                19.0       2310.0           471.0
  ⋮   │     ⋮         ⋮              ⋮                ⋮             ⋮          ⋱
 2994 │   -117.91     33.6         

In [17]:
@btime DuckDB.execute(con, "SELECT * FROM  CALHOUSETST where total_rooms > 2000.0 ")

  222.166 μs (50 allocations: 1.92 KiB)


1620×9 DataFrame
  Row │ longitude  latitude  housing_median_age  total_rooms  total_bedrooms   ⋯
      │ Float64?   Float64?  Float64?            Float64?     Float64?         ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │   -122.05     37.37                27.0       3885.0           661.0   ⋯
    2 │   -117.81     33.78                27.0       3589.0           507.0
    3 │   -120.65     35.48                19.0       2310.0           471.0
    4 │   -122.84     38.4                 15.0       3080.0           617.0
    5 │   -118.02     34.08                31.0       2402.0           632.0   ⋯
    6 │   -117.03     32.97                16.0       3936.0           694.0
    7 │   -117.97     33.73                27.0       2097.0           325.0
    8 │   -121.2      38.69                26.0       3077.0           607.0
  ⋮   │     ⋮         ⋮              ⋮                ⋮             ⋮          ⋱
 1614 │   -122.54     37.9         

In [58]:
@btime DuckDB.execute(con, "SELECT * FROM  DATA1  WHERE c> 600000")

  2.131 ms (24 allocations: 816 bytes)


399901×3 DataFrame
    Row │ a        b          c       
        │ Int32?   Float64?   Int32?  
────────┼─────────────────────────────
      1 │  600100  0.114716    600100
      2 │  600101  0.342306    600100
      3 │  600102  0.854904    600100
      4 │  600103  0.25516     600100
      5 │  600104  0.294107    600100
      6 │  600105  0.774148    600100
      7 │  600106  0.498302    600100
      8 │  600107  0.647357    600100
   ⋮    │    ⋮         ⋮         ⋮
 399895 │  999994  0.598799    999900
 399896 │  999995  0.666       999900
 399897 │  999996  0.951518    999900
 399898 │  999997  0.415549    999900
 399899 │  999998  0.588543    999900
 399900 │  999999  0.45919     999900
 399901 │ 1000000  0.135341   1000000
                   399886 rows omitted

In [56]:
DuckDB.execute(con, "CREATE INDEX IDX1 ON DATA1 (a,c)")

0×1 DataFrame
 Row │ Count 
     │ Int64 
─────┴───────

In [60]:
@btime DuckDB.execute(con, "SELECT sum(b*a) FROM  DATA1  WHERE c=600100")

  222.917 μs (16 allocations: 656 bytes)


1×1 DataFrame
 Row │ sum((b * a)) 
     │ Float64?     
─────┼──────────────
   1 │    3.02259e7

In [5]:
close(con)